# Error Handling Example

This script shows how handle errors occurring
during API calls to the
Vibration service

The mvg design principle is that
the client application using the MVG
API is responsible for error handling

In [1]:
import json
import sys
import logging
from requests import HTTPError

# import mvg library with python bindings to mvg-API
from mvg import MVG

Note that the `TOKEN` is used both for authorization and authentication. Thus, each unique token represents a unique user and each user has their own unique database on the VA vibration service.

**You need to insert your token received from Viking Analytics here:**

In [2]:
TOKEN = "NO TOKEN"  # replace with your token

This is Viking Analytics default logging setup and can be adapted to suit your needs. Log messages are printed from `mvg` library, see the source code for details.

In [3]:
root_logger = logging.getLogger()
root_logger.setLevel("INFO")
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(name)s - %(message)s")
stream_handler = logging.StreamHandler(stream=sys.stderr)
stream_handler.setFormatter(formatter)
root_logger.addHandler(stream_handler)

As we are interested in error handling we set the logger level to `logging.DEBUG`. That will trace out the detailed information on exceptions provided by the server.


Set log level to `logging.ERROR` to just see the results of proper error handling in the code `root_logger.setLevel(logging.ERROR)`

In [4]:
root_logger.setLevel(logging.DEBUG)

Instantiate a session object with mvg library a session object basically caches the endpoint and the token, to simplify the the calls to the mvg library.

In [5]:
ENDPOINT = "https://api.beta.multiviz.com"
session = MVG(ENDPOINT, TOKEN)

2021-03-18 11:13:44,009 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): api.beta.multiviz.com:443
2021-03-18 11:13:44,137 - DEBUG - urllib3.connectionpool - https://api.beta.multiviz.com:443 "GET / HTTP/1.1" 200 465


We now check if the server is alive. The hello message contains, amongst others the API version.

In [6]:
hello_message = json.dumps(session.say_hello())
print(hello_message)

2021-03-18 11:13:44,141 - INFO - mvg.mvg - Getting API info for: https://api.beta.multiviz.com
2021-03-18 11:13:44,143 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): api.beta.multiviz.com:443
2021-03-18 11:13:44,269 - DEBUG - urllib3.connectionpool - https://api.beta.multiviz.com:443 "GET / HTTP/1.1" 200 465


{"api": {"name": "MultiViz Engine API", "version": "v0.0.13"}, "features": {"mode_id": true, "blacksheep": false, "emerging_mode": false, "on_off": false, "asset_type": false, "indicator_arrow": false}, "request_status": ["initialized", "accepted", "queued", "ongoing", "failed", "ready"], "traffic_limit": 10000000, "max_source_id_length": 255, "max_meta_size": 50000, "max_number_of_samples": 400000, "max_number_of_sources": 100, "max_number_of_measurements": 1000, "db_version": "1.0.0"}


## Invalid Token

Lets provoke an error by creating a session with a non-valid token.

In [7]:
unauth_session = MVG(ENDPOINT, "PASSKEY")

2021-03-18 11:13:44,274 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): api.beta.multiviz.com:443
2021-03-18 11:13:44,393 - DEBUG - urllib3.connectionpool - https://api.beta.multiviz.com:443 "GET / HTTP/1.1" 200 465


We need to call a method requiring authentication. Check http://endpoint/docs to see which methods require authentication, they have a lock icon.

In [8]:
try:
    print(">>> Provoke Unathorized call")
    sources = unauth_session.list_sources()
except HTTPError as exc:
    # As we have rouge token
    # We'll end up here
    print("OUCH")
    # The exception will be risen by
    # raise_for_status from the requests
    # We'll print it and see that it
    # states an authorization error (401)
    print(exc)

2021-03-18 11:13:44,398 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 11:13:44,399 - INFO - mvg.mvg - listing sources
2021-03-18 11:13:44,400 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): api.beta.multiviz.com:443


>>> Provoke Unathorized call


2021-03-18 11:13:44,544 - DEBUG - urllib3.connectionpool - https://api.beta.multiviz.com:443 "GET /sources/ HTTP/1.1" 401 25
2021-03-18 11:13:44,546 - DEBUG - mvg.mvg - 401 Client Error: Unauthorized for url: https://api.beta.multiviz.com/sources/
2021-03-18 11:13:44,546 - DEBUG - mvg.mvg - {"detail":"Unauthorized"}


OUCH
401 Client Error: Unauthorized for url: https://api.beta.multiviz.com/sources/


We'll now return to the original session and show how to get details when the server internally rejected a request for other reasons.

## Illegal source_id

We provoke the service using an illegal string for the `source_id`

In [9]:
try:
    print(">>> Provoke illegal source Id name")
    session.create_source("illegal%name", {})
except HTTPError as exc:
    # As we have rouge token
    # We'll end up here
    print("OUCH")
    # The exception will be risen by
    # raise_for_status from the requests
    # We'll print it and see that it
    # states an authorization error (401)
    print(exc)

2021-03-18 11:13:44,551 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 11:13:44,551 - INFO - mvg.mvg - creating source with source id=illegal%name
2021-03-18 11:13:44,552 - INFO - mvg.mvg - metadata: {}
2021-03-18 11:13:44,553 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): api.beta.multiviz.com:443


>>> Provoke illegal source Id name


2021-03-18 11:13:44,683 - DEBUG - urllib3.connectionpool - https://api.beta.multiviz.com:443 "POST /sources/ HTTP/1.1" 401 25
2021-03-18 11:13:44,684 - DEBUG - mvg.mvg - 401 Client Error: Unauthorized for url: https://api.beta.multiviz.com/sources/
2021-03-18 11:13:44,685 - DEBUG - mvg.mvg - {"detail":"Unauthorized"}


OUCH
401 Client Error: Unauthorized for url: https://api.beta.multiviz.com/sources/


We'll show a couple of examples what can go wrong when creating a measurement

## Non-existing Source

In some cases where there is detailed information provided by the server, we can retrieve it by inspecting the exception object `exc`, which actually contains all information about the request.

In [10]:
try:
    print(">>> Non existing source")
    d = [1, 2, 3]
    session.create_measurement(sid="",
                               duration=-3,
                               timestamp=-5,
                               data=d,
                               meta={})
except HTTPError as exc:
    print(exc)
    print("Details on error")
    print(exc.response.json())

2021-03-18 11:13:44,689 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 11:13:44,689 - INFO - mvg.mvg - creating measurement from source id=
2021-03-18 11:13:44,690 - INFO - mvg.mvg -   duration:  -3
2021-03-18 11:13:44,690 - INFO - mvg.mvg -   timestamp: -5
2021-03-18 11:13:44,691 - INFO - mvg.mvg -   meta data: {}
2021-03-18 11:13:44,692 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): api.beta.multiviz.com:443


>>> Non existing source


2021-03-18 11:13:44,825 - DEBUG - urllib3.connectionpool - https://api.beta.multiviz.com:443 "POST /sources//measurements HTTP/1.1" 404 22
2021-03-18 11:13:44,826 - DEBUG - mvg.mvg - 404 Client Error: Not Found for url: https://api.beta.multiviz.com/sources//measurements
2021-03-18 11:13:44,827 - DEBUG - mvg.mvg - {"detail":"Not Found"}


404 Client Error: Not Found for url: https://api.beta.multiviz.com/sources//measurements
Details on error
{'detail': 'Not Found'}


## Parameter Value Out of Range

In [11]:
try:
    print(">>> Parameter value out of range")
    d = [1, 2, 3]
    session.create_measurement(sid="u0001",
                               duration=-3,
                               timestamp=-5,
                               data=d,
                               meta={})
except HTTPError as exc:
    print(exc)
    # Retrieve details
    print("Details on error")
    print(exc.response.json())

2021-03-18 11:13:44,832 - INFO - mvg.mvg - endpoint https://api.beta.multiviz.com
2021-03-18 11:13:44,832 - INFO - mvg.mvg - creating measurement from source id=u0001
2021-03-18 11:13:44,833 - INFO - mvg.mvg -   duration:  -3
2021-03-18 11:13:44,833 - INFO - mvg.mvg -   timestamp: -5
2021-03-18 11:13:44,834 - INFO - mvg.mvg -   meta data: {}
2021-03-18 11:13:44,835 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): api.beta.multiviz.com:443


>>> Parameter value out of range


2021-03-18 11:13:44,964 - DEBUG - urllib3.connectionpool - https://api.beta.multiviz.com:443 "POST /sources/u0001/measurements HTTP/1.1" 401 25
2021-03-18 11:13:44,966 - DEBUG - mvg.mvg - 401 Client Error: Unauthorized for url: https://api.beta.multiviz.com/sources/u0001/measurements
2021-03-18 11:13:44,966 - DEBUG - mvg.mvg - {"detail":"Unauthorized"}


401 Client Error: Unauthorized for url: https://api.beta.multiviz.com/sources/u0001/measurements
Details on error
{'detail': 'Unauthorized'}


## Missing Parameter

In this case we have an error on the client side. This means that the data is never sent to the server so we do not get an `HTTPError`, but a `TypeError` instead. Only the `HTTPError` contains any details from the server, so to be on the safe side we should catch them separately.

In [12]:
try:
    print(">>> Parameter missing in mvg call")
    session.create_measurement(sid="u0001",
                               duration=-3,
                               timestamp=-5,
                               meta={})
except HTTPError as exc:
    print(exc)
    print("Details on error")
    print(exc.response.json())
except Exception as exc:
    print(f"{type(exc).__name__}: {exc}")
    print("No details from server available")

>>> Parameter missing in mvg call
TypeError: create_measurement() missing 1 required positional argument: 'data'
No details from server available
